In [1]:
import numpy as np
import pandas as pd
from itertools import product, permutations, combinations, combinations_with_replacement
from sklearn.metrics import precision_recall_curve, roc_curve, auc

In [2]:
def outDF(model = "non_filter",hvg_version = "sct",data_version = "count",gene_filter = False, directory = "/home/zc354/GRN/output/network_2000/",true_path ='/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv' ):
    pred_path=directory+model+"_"+hvg_version+"_"+data_version+'.csv'
    
    trueEdgesDF = pd.read_csv(true_path,sep = ',', header = 0, index_col = None)
    predEdgesDF = pd.read_csv(pred_path, sep = ',', header =  0, index_col = None)
    
    if(gene_filter == True):
        hvg = pd.read_csv('/gpfs/gibbs/pi/zhao/zc354/GRN/output/hvg_bmmc_3000.csv',sep = ',', header = 0, index_col = None)
        hvgset = set(hvg['x'])
        predEdgesDF = predEdgesDF[(predEdgesDF['Gene2'].isin(hvgset))]
                                  
    print('length of pred-network:'+str(len(predEdgesDF['Gene1'])))
    
    predEdgesDF.drop(predEdgesDF[predEdgesDF['Edgeweight'] == 0].index,inplace=True)

    trueEdgesDF = trueEdgesDF.loc[(trueEdgesDF['Gene1'] != trueEdgesDF['Gene2'])]
    predEdgesDF = predEdgesDF.loc[(predEdgesDF['Gene1'] != predEdgesDF['Gene2'])]

    trueEdgesDF.drop_duplicates(keep = 'first', inplace=True)
    predEdgesDF.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)

    print('length of true-network TF:'+str(len(set(trueEdgesDF['Gene1']))))
    print('length of true-network gene:'+str(len(set(trueEdgesDF['Gene2']))))
    print('length of pred-network TF:'+str(len(set(predEdgesDF['Gene1']))))
    print('length of pred-network gene:'+str(len(set(predEdgesDF['Gene2']))))

    true_TF = set(trueEdgesDF['Gene1'])
    true_tg = set(trueEdgesDF['Gene2'])

    pred_TF = set(predEdgesDF['Gene1'])
    pred_tg = set(predEdgesDF['Gene2'])

    TFset = (true_TF & pred_TF) 
    tgset = (true_tg & pred_tg)
    
    print('length of TF_intersect:'+str(len(TFset)))
    print('length of tg_intersect:'+str(len(tgset)))

    trueEdgesDF = trueEdgesDF[(trueEdgesDF['Gene1'].isin(TFset))&(trueEdgesDF['Gene2'].isin(tgset))]
    predEdgesDF = predEdgesDF[(predEdgesDF['Gene1'].isin(TFset))&(predEdgesDF['Gene2'].isin(tgset))]
    print('length of TF in truenetwork:'+str(len(set(trueEdgesDF['Gene1'])))+'_'+'length of tg in truenetwork:'+str(len(set(trueEdgesDF['Gene2']))))
    print('length of TF in prednetwork:'+str(len(set(predEdgesDF['Gene1'])))+'_'+'length of tg in prednetwork:'+str(len(set(predEdgesDF['Gene2']))))
    
    #possibleEdges = set(product(set(trueEdgesDF['Gene1']),set(trueEdgesDF['Gene2'])))
    possibleEdges = set(product(TFset,tgset))
    possibleEdgesDF = pd.DataFrame(list(possibleEdges))
    possibleEdgesDF.columns=['Gene1','Gene2']
    
    TrueEdgesDF = pd.DataFrame(columns=['Edge'])
    TrueEdgesDF['Edge'] = trueEdgesDF['Gene1'].map(str) +"|"+ trueEdgesDF['Gene2'].map(str)
    PredEdgesDF = pd.DataFrame(columns=['Edge','Weight'])
    PredEdgesDF['Edge'] = predEdgesDF['Gene1'].map(str) +"|"+ predEdgesDF['Gene2'].map(str)
    PredEdgesDF['Weight'] = predEdgesDF['Edgeweight']
    
    PossibleEdgesDF = pd.DataFrame(columns=['Edge','Pred','True'])
    PossibleEdgesDF['Edge'] = possibleEdgesDF['Gene1'].map(str) +"|"+ possibleEdgesDF['Gene2'].map(str)
    
    #Generate the set of true/pred/possible edges
    TrueEdgesset=set(TrueEdgesDF['Edge'])
    PredEdgesset=set(PredEdgesDF['Edge'])
    PossibleEdgesset=set(PossibleEdgesDF['Edge'])
    
    print("length of trueEdges:"+str(len(TrueEdgesset)))
    print("length of predEdges:"+str(len(PredEdgesset)))
    print("length of backgroudEdges:"+str(len(PossibleEdgesset)))
    #Generate four possible conditions using set operation
    #inbothset represents edges in both true and pred network
    #onlyintrue set represents edges in true network but not in pred
    #onlyinpred set represents edges in pred but not in true network
    #inneitherset represent edges in neither of true and pred network but in possible network
    inbothset = (TrueEdgesset & PredEdgesset) & PossibleEdgesset
    onlyintrue = (TrueEdgesset - PredEdgesset) & PossibleEdgesset
    onlyinpred = (PredEdgesset - TrueEdgesset) & PossibleEdgesset
    inneitherset = PossibleEdgesset - (TrueEdgesset | PredEdgesset)
    print('length of inbothset:'+str(len(inbothset)))
    print('length of onlyintrue:'+str(len(onlyintrue)))
    print('length of onlyinpred:'+str(len(onlyinpred)))
    print('length of inneitherset:'+str(len(inneitherset)))
    #Set Edges as index in three DataFrames to facilitate indexing
    PredEdgesDF.set_index('Edge',inplace=True)
    TrueEdgesDF.set_index('Edge',inplace=True)
    PossibleEdgesDF.set_index('Edge',inplace=True)
    
    #Assign values to different types of edges
    inbothDF = PredEdgesDF.loc[list(inbothset),:]
    inbothDF['True'] = 1
    inbothDF.columns = ['Pred','True']

    onlyintrueDF = TrueEdgesDF.loc[list(onlyintrue),:]
    onlyintrueDF['Pred'] = 0
    onlyintrueDF['True'] = 1

    onlyinpredDF = PredEdgesDF.loc[list(onlyinpred),:]
    onlyinpredDF['True'] = 0
    onlyinpredDF.columns = ['Pred','True']

    inneitherDF = PossibleEdgesDF.loc[list(inneitherset),:]
    inneitherDF['Pred'] = 0
    inneitherDF['True'] = 0

    outDF = pd.concat([inbothDF,onlyintrueDF,onlyinpredDF,inneitherDF],axis=0)
    return PredEdgesDF,outDF

In [7]:
p_value_cutoff=0.05
score_cutoff=0.1
expression_form="SCT"
link_signal = "all"
data_version = "count"
gene_peak = "linkpeaks"
data_version = data_version+"_bmmc_"+gene_peak+"_motifmatchr"
#data_version = data_version+"_motifmatchr"
#data_version_2 = "norm"+"_linkpeaks_"+str(p_value_cutoff)+"_"+str(score_cutoff)+"_"+str(expression_form)+"_"+link_signal

In [8]:
data_version

'count_bmmc_linkpeaks_motifmatchr'

In [9]:
PredEdgesDF_non_filter,outDF_non_filter = outDF(model = "non_filter",hvg_version = "sct",data_version = data_version)

length of pred-network:1043500
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:1608
length of TF_intersect:370
length of tg_intersect:1440
length of TF in truenetwork:261_length of tg in truenetwork:1362
length of TF in prednetwork:370_length of tg in prednetwork:1440
length of trueEdges:10879
length of predEdges:532715
length of backgroudEdges:532800
length of inbothset:10879
length of onlyintrue:0
length of onlyinpred:521836
length of inneitherset:85


In [10]:
PredEdgesDF_non_filter_2,outDF_non_filter_2 = outDF(model = "non_filter",hvg_version = "sct",data_version = data_version,gene_filter = True)

length of pred-network:1043500
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:1608
length of TF_intersect:370
length of tg_intersect:1440
length of TF in truenetwork:261_length of tg in truenetwork:1362
length of TF in prednetwork:370_length of tg in prednetwork:1440
length of trueEdges:10879
length of predEdges:532715
length of backgroudEdges:532800
length of inbothset:10879
length of onlyintrue:0
length of onlyinpred:521836
length of inneitherset:85


In [11]:
PredEdgesDF_filter,outDF_filter = outDF(model = "filter",hvg_version = "sct",data_version = data_version)

length of pred-network:277467
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:984
length of TF_intersect:370
length of tg_intersect:919
length of TF in truenetwork:252_length of tg in truenetwork:874
length of TF in prednetwork:370_length of tg in prednetwork:919
length of trueEdges:7671
length of predEdges:138652
length of backgroudEdges:340030
length of inbothset:3567
length of onlyintrue:4104
length of onlyinpred:135085
length of inneitherset:197274


In [ ]:
PredEdgesDF_filter_2,outDF_filter_2 = outDF(model = "filter",hvg_version = "sct",data_version = data_version,gene_filter = True)

In [ ]:
PredEdgesDF_multiply,outDF_multiply = outDF(model = "multiply",hvg_version = "sct",data_version =data_version)

In [ ]:
PredEdgesDF_multiply_2,outDF_multiply_2 = outDF(model = "multiply",hvg_version = "sct",data_version =data_version,gene_filter = True)

In [ ]:
PredEdgesDF_multiply,outDF_multiply = outDF(model = "multiply",hvg_version = "sct",data_version =data_version)

In [17]:
PredEdgesDF_multiply_2,outDF_multiply_2 = outDF(model = "multiply",hvg_version = "sct",data_version =data_version,gene_filter = True)

length of pred-network:941439
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:1589
length of TF_intersect:370
length of tg_intersect:1436
261_1358
370_1436
length of trueEdges:10832
length of predEdges:473248
length of backgroudEdges:531320
length of inbothset:9911
length of onlyintrue:921
length of onlyinpred:463337
length of inneitherset:57151


In [18]:
PredEdgesDF_score,outDF_score = outDF(model = "score",hvg_version = "sct",data_version = data_version)

length of pred-network:921434
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:1589
length of TF_intersect:370
length of tg_intersect:1436
261_1358
370_1436
length of trueEdges:10832
length of predEdges:473222
length of backgroudEdges:531320
length of inbothset:9912
length of onlyintrue:920
length of onlyinpred:463310
length of inneitherset:57178


In [19]:
PredEdgesDF_score_2,outDF_score_2 = outDF(model = "score",hvg_version = "sct",data_version = data_version,gene_filter = True)

length of pred-network:921434
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF:649
length of pred-network gene:1589
length of TF_intersect:370
length of tg_intersect:1436
261_1358
370_1436
length of trueEdges:10832
length of predEdges:473222
length of backgroudEdges:531320
length of inbothset:9912
length of onlyintrue:920
length of onlyinpred:463310
length of inneitherset:57178


In [20]:
prec_non_filter, recall_non_filter, thresholds_non_filter = precision_recall_curve(y_true=outDF_non_filter ['True'],probas_pred=outDF_non_filter ['Pred'], pos_label=1)
prec_filter, recall_filter, thresholds_filter = precision_recall_curve(y_true=outDF_filter ['True'],probas_pred=outDF_filter ['Pred'], pos_label=1)
prec_multiply, recall_multiply, thresholds_multiply = precision_recall_curve(y_true=outDF_multiply['True'],probas_pred=outDF_multiply['Pred'], pos_label=1)
prec_score, recall_score, thresholds_score = precision_recall_curve(y_true=outDF_score['True'],probas_pred=outDF_score['Pred'], pos_label=1)

In [21]:
prec_non_filter_2, recall_non_filter_2, thresholds_non_filter_2 = precision_recall_curve(y_true=outDF_non_filter_2 ['True'],probas_pred=outDF_non_filter_2 ['Pred'], pos_label=1)
prec_filter_2, recall_filter_2, thresholds_filter_2 = precision_recall_curve(y_true=outDF_filter_2 ['True'],probas_pred=outDF_filter_2 ['Pred'], pos_label=1)
prec_multiply_2, recall_multiply_2, thresholds_multiply_2 = precision_recall_curve(y_true=outDF_multiply_2['True'],probas_pred=outDF_multiply_2['Pred'], pos_label=1)
prec_score_2, recall_score_2, thresholds_score_2 = precision_recall_curve(y_true=outDF_score_2['True'],probas_pred=outDF_score_2['Pred'], pos_label=1)

In [22]:
print('non_filter_auprc:'+str(auc(recall_non_filter,prec_non_filter)))
print('filter_auprc:'+str(auc(recall_filter,prec_filter)))
print('multiply_auprc:'+str(auc(recall_multiply,prec_multiply)))
print('score_auprc:'+str(auc(recall_score,prec_score)))

non_filter_auprc:0.02831592504293671
filter_auprc:0.028099858753709312
multiply_auprc:0.028191021811489785
score_auprc:0.028252300396028183


In [23]:
print('non_filter_auprc:'+str(auc(recall_non_filter_2,prec_non_filter_2)))
print('filter_auprc:'+str(auc(recall_filter_2,prec_filter_2)))
print('multiply_auprc:'+str(auc(recall_multiply_2,prec_multiply_2)))
print('score_auprc:'+str(auc(recall_score_2,prec_score_2)))

non_filter_auprc:0.02831592504293671
filter_auprc:0.028099858753709312
multiply_auprc:0.028191021811489785
score_auprc:0.028252300396028183


In [ ]:
recall_multiply[0:100]

In [28]:
pd.DataFrame(recall_multiply)

,0
0,1.00000
1,0.59042
2,0.59042
3,0.59042
4,0.59042
...,...
342718,0.00000
342719,0.00000
342720,0.00000
342721,0.00000


In [27]:
pd.DataFrame(recall_non_filter)

,0
0,1.000000
1,0.995329
2,0.995329
3,0.995329
4,0.995329
...,...
199595,0.000167
199596,0.000000
199597,0.000000
199598,0.000000


In [78]:
topk_multiply = pd.DataFrame(thresholds_multiply)

In [79]:
topk = np.zeros(len(thresholds_multiply))
for i in range(0,len(thresholds_multiply)):
    topk[i]=len(PredEdgesDF_multiply.loc[(PredEdgesDF_multiply['Weight'] >= thresholds_multiply[i])])

In [80]:
topk_multiply['k']=topk
topk_multiply['precision']=prec_multiply[0:32445]
topk_multiply['recall']=pd.DataFrame(recall_multiply)
topk_multiply.columns = ['threshold','k','precision','recall']

In [85]:
topk_multiply['precision'][1:len(prec_multiply)] = prec_multiply[0:len(prec_multiply)-2]

In [87]:
topk_multiply[32400:32445]

,threshold,k,precision,recall
32400,0.213556,56.0,0.152174,0.001319
32401,0.215094,55.0,0.155556,0.001130
32402,0.218265,54.0,0.136364,0.000942
32403,0.218795,53.0,0.116279,0.000942
32404,0.220388,52.0,0.119048,0.000942
32405,0.222503,51.0,0.121951,0.000942
32406,0.222543,50.0,0.125000,0.000753
32407,0.223824,48.0,0.102564,0.000753
32408,0.231038,47.0,0.105263,0.000753
32409,0.234010,46.0,0.108108,0.000753


In [88]:
topk_multiply_10 = topk_multiply[topk_multiply.k % 10 ==0]
topk_multiply_10.sort_values(by=['k'])

,threshold,k,precision,recall
32436,3.800474e-01,10.0,0.000000,0.000000
32428,3.255091e-01,20.0,0.000000,0.000000
32414,2.501784e-01,40.0,0.062500,0.000377
32406,2.225428e-01,50.0,0.125000,0.000753
32397,2.094634e-01,60.0,0.163265,0.001507
...,...,...,...,...
43,2.300526e-08,58020.0,0.044004,0.268600
36,1.768529e-08,58030.0,0.043995,0.268600
30,1.119356e-08,58040.0,0.044017,0.268600
19,4.140573e-09,58060.0,0.044002,0.268789


In [38]:
topk_2 = np.zeros(len(thresholds_non_filter))
for i in range(0,len(thresholds_non_filter)):
    topk_2[i]=len(PredEdgesDF_non_filter.loc[(PredEdgesDF_non_filter['Weight'] >= thresholds_non_filter[i])])
    if i%1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [37]:
len(thresholds_non_filter)

199599

In [39]:
topk_non_filter = pd.DataFrame(thresholds_non_filter)

In [ ]:
prec_non_filter[]

In [62]:
topk_non_filter['k']=topk_2
topk_non_filter['precision']=prec_non_filter[0:len(prec_non_filter)-1]
topk_non_filter['recall']=recall_non_filter[1:len(recall_non_filter)]
topk_non_filter.columns = ['threshold','k','precision','recall']

In [74]:
topk_non_filter['precision'][1:len(topk_non_filter)] = prec_non_filter[0:len(prec_non_filter)-2]

In [75]:
topk_non_filter[len(topk_non_filter)-20:len(topk_non_filter)]

,threshold,k,precision,recall
199579,0.186472,21.0,0.190476,0.000501
199580,0.187355,20.0,0.200000,0.000501
199581,0.189841,19.0,0.157895,0.000501
199582,0.199605,18.0,0.166667,0.000501
199583,0.208097,17.0,0.176471,0.000501
199584,0.209139,16.0,0.187500,0.000501
199585,0.209947,15.0,0.200000,0.000501
199586,0.215769,14.0,0.214286,0.000501
199587,0.216788,13.0,0.230769,0.000501
199588,0.219916,12.0,0.250000,0.000501


In [89]:
topk_non_filter.to_csv("/gpfs/gibbs/pi/zhao/zc354/GRN/output/topk_non_filter.csv",index=False,sep=',')

In [91]:
topk_multiply.to_csv("/gpfs/gibbs/pi/zhao/zc354/GRN/output/topk_multiply.csv",index=False,sep=',')

In [52]:
topk_non_filter_10 = topk_non_filter[topk_non_filter.k % 10 ==0]

In [92]:
topk_multiply_10.to_csv("/gpfs/gibbs/pi/zhao/zc354/GRN/output/topk_multiply_10.csv",index=False,sep=',')

In [93]:
topk_non_filter_10.to_csv("/gpfs/gibbs/pi/zhao/zc354/GRN/output/topk_non_filter_10.csv",index=False,sep=',')

In [59]:
topk_non_filter = pd.DataFrame(thresholds_non_filter)

In [61]:
topk_non_filter.columns = ['thresholds']

In [ ]:
topk_non_filter['k']=topk